# Lab - Classification

In this lab, we are going to build a classification module. When given an image of a handwritten digit like the one below, the model will be able to tell which digit is in the image.

<img src='test2.jpg'>

In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn import svm
import numpy as np
import pickle 
import argparse
import imutils  # If you are unable to install this library, ask the TA; we only need this in extract_hsv_histogram.
import cv2
import os
import random


# Depending on library versions on your system, one of the following imports 
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split

In [22]:
path_to_dataset = r'digits_dataset'
target_img_size = (32, 32) # fix image size because classification algorithms THAT WE WILL USE HERE expect that

# We are going to fix the random seed to make our experiments reproducible 
# since some algorithms use pseudorandom generators
random_seed = 42  
random.seed(random_seed)
np.random.seed(random_seed)

In [23]:
def extract_features(img):
   
    img = cv2.resize(img,target_img_size)
    win_size = (32, 32)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2)
    
    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 9  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten()

The following function will extract the features and the label of each image in our dataset and save it in RAM. We normally don't save datasets in RAM, but this dataset is small.

In [24]:
def load_dataset():
    features = []
    labels = []
    img_filenames = os.listdir(path_to_dataset)

    for i, fn in enumerate(img_filenames):
        if fn.split('.')[-1] != 'jpg':
            continue

        label = fn.split('.')[0]
        labels.append(label)

        path = os.path.join(path_to_dataset, fn)
        img = cv2.imread(path)
        features.append(extract_features(img))
        
        # show an update every 1,000 images
        if i > 0 and i % 1000 == 0:
            print("[INFO] processed {}/{}".format(i, len(img_filenames)))
        
    return features, labels        

In [25]:
classifiers = {
   'KNN': KNeighborsClassifier(n_neighbors=9),

}

In [26]:
# This function will test all our classifiers on a specific feature set
def run_experiment():
    
    # Load dataset with extracted features
    print('Loading dataset. This will take time ...')
    features, labels = load_dataset()
    print('Finished loading dataset.')
    
    # Since we don't want to know the performance of our classifier on images it has seen before
    # we are going to withhold some images that we will test the classifier on after training 
    train_features, test_features, train_labels, test_labels = train_test_split(
        features, labels, test_size=0.2, random_state=random_seed)
    
    desired_classifier = 'KNN'

    # Check if the desired classifier is in the dictionary
    if desired_classifier in classifiers:
        print('############## Training', desired_classifier, "##############")

        # Get the model for the desired classifier
        model = classifiers[desired_classifier]

        # Train the model only on the training features
        model.fit(train_features, train_labels)

        # Test the model on images it hasn't seen before
        accuracy = model.score(test_features, test_labels)

        print(desired_classifier, 'accuracy:', accuracy * 100, '%')

        # Save the trained model to a file
        with open('model.pkl', 'wb') as file:
            pickle.dump(model, file)
        
        print('Trained model saved as model.pkl')
    else:
        print(f"The desired classifier '{desired_classifier}' is not in the classifiers dictionary.")


Now, we see how each classifier and each feature set performs

In [27]:
run_experiment()


Loading dataset. This will take time ...
[INFO] processed 1000/7200
[INFO] processed 2000/7200
[INFO] processed 3000/7200
[INFO] processed 4000/7200
[INFO] processed 5000/7200
[INFO] processed 6000/7200
[INFO] processed 7000/7200
Finished loading dataset.
############## Training KNN ##############
KNN accuracy: 96.18055555555556 %
Trained model saved as model.pkl


The classifiers list now has models trained on the last feature set you ran an experiment on. You can play around with it checking the probability it gives to each label, given an image.

In [28]:



# Example
# test_img_path = r'test2.jpg'
# img = cv2.imread(test_img_path)
# raw_features = extract_features(img)  # be careful of the choice of feature set


In [29]:
nn = classifiers['SVM']
nn.predict_proba([raw_features])

KeyError: 'SVM'

Try to get a better accuracy by changing the model hyperparameters and retraining.

Well done! You have now finished the Image Processing & Computer Vision course. Study well for the final exam 😄. If you have any questions, don't hesitate to ask now.